In [11]:
from datasets import load_dataset, load_from_disk
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
import aiohttp

Only uncomment loading and saving if we want to overwrite dataset.hf

In [12]:
# dataset = load_dataset("open_subtitles", lang1="en", lang2="es", trust_remote_code=True,
#            storage_options={'client_kwargs': {'timeout': aiohttp.ClientTimeout(total=3600)}}, num_proc= 8, split=None)

In [13]:
# dataset.save_to_disk("dataset.hf")

In [14]:
dataset = load_from_disk("dataset.hf")
split_dataset = dataset["train"].train_test_split(train_size=0.8, seed=20)

Only uncomment if we want to retrain the Mbart tokenizer

In [15]:
# tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="es_XX")

In [16]:
# max_length = 512
# 
# def preprocess_text(text):
#     inputs = [txt["en"] for txt in text["translation"]]
#     targets = [txt["es"] for txt in text["translation"]]
#     model_inputs = tokenizer(
#         inputs, text_target=targets, max_length=max_length, truncation=True
#     )
#     return model_inputs

In [17]:
# tokenized_datasets = split_dataset.map(
#     preprocess_text,
#     batched=True,
#     remove_columns=split_dataset["train"].column_names,
# )

In [18]:
#tokenized_datasets.save_to_disk("tokenized_dataset.hf")

In [19]:
import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize
import csv

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marcogracie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Adding linear regression tokenizer
def tokenize_batch(batch):
    return {"tokenized": [word_tokenize(txt["en"]) for txt in batch["translation"]]}

# Apply tokenization to the dataset and save
tokenized_train = split_dataset["train"].map(tokenize_batch, batched=True)
tokenized_train.to_csv("encoded_inputs_train.csv")

tokenized_test = split_dataset["test"].map(tokenize_batch, batched=True)
tokenized_test.to_csv("encoded_inputs_test.csv")

Map:   0%|          | 0/49147400 [00:00<?, ? examples/s]

In [ ]:
# loading csvs
# with open("encoded_inputs_train.csv", "r") as f:
#     reader = csv.reader(f)
#     encoded_inputs_train = [list(map(int, row)) for row in reader]